In [519]:
import json
#from functools import partial
import functools
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left
import re
import gzip
import zlib
def checker(nm,ori):
    flag = False
    for key in ori.keys():
        if ori[key] != nm[key]:
            flag = True
            break
    print('Match!!' if flag == False else 'Error!!')

def test(origin_mapping,idx):
    print(len(idx))
    mark = consecu_int_run_length(idx)
    # check part
    _m = mark
    nm = {}
    for i in range(len(keys)):
        nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    checker(nm,origin_mapping)

    sporadic_seg = sporadic_seg_run_length(mark)
    # check part
    _m = sporadic_seg
    nm = {}
    vv = set(values)
    for i in range(len(keys)):
        if keys[i] in values:
            nm[keys[i]] = keys[i]
        #nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        if _m[i+2] != 1:
            for j in range(_m[i+2]):
                nm[keys[_m[i+1]+j]] = values[_m[i]+j]
        else:
            length = len(_m[i+1])
            for j in range(length):
                nm[keys[_m[i+1][j]]] = values[_m[i]+j]
    checker(nm,origin_mapping)

def consecu_int_run_length(idx):
    mark = []
    flag = False
    temp = []
    for i in range(len(idx)):
        if i != idx[i]:      
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                temp = []
                flag = False
    if len(temp) != 0:
        mark += temp
    return mark

def sporadic_seg_run_length(mark):
    sporadic_seg = [ mark[i:i+3] for i in range(0,len(mark),3)]
    flag = False
    temp = []
    res = []
    for i in range(len(sporadic_seg)):
        if sporadic_seg[i][2] != 1:
            if flag == True:
                res += temp 
                # initialize temp to an empty list, to avoid if temp have something but index is at the end
                temp = []
                flag = False
            res += sporadic_seg[i]
        else:
            # first one
            if flag == False:
                # start index, index list, len
                temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
                flag = True
            else:
                if sporadic_seg[i][0] - sporadic_seg[i-1][0] == 1:
                    temp[1].append(sporadic_seg[i][1]) 
                else:
                    res += temp  
                    temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
    if len(temp) != 0:
        res += temp  
    return res

def classify(origin_mapping):
    same, prefix, infix, postfix, others = [], [], [], [], []
    for k,v in origin_mapping.items():
        index = v.find(k)
        if k == v:
            same += [k]
        elif index != -1:
            if index == 0:
                prefix += [k]
            elif v[index:] == k:
                postfix += [k]
            else:
                infix += [k]
        else:
            others += [k]
    print('same: {}\nprefix: {}\ninfix: {}\npostfix: {}\nothers: {}'.format(len(same),len(prefix),len(infix),len(postfix),len(others)))
    return same, prefix, infix, postfix, others
g=len
def p(x,y):
    d,b=g(x[0]),g(y[0])
    if x[1]==y[1]:
        if d-b:
            return 1 if d>b else -1
        else:
            for i in range(d):
                if x[0][i]==y[0][i]:
                    continue
                return 1 if int(x[0][i])>int(y[0][i]) else -1       
            return 0
    return 1 if x[1]>y[1] else -1
def t(l):
    r,e=[],[]
    for i in range(g(l)):
        y = re.findall('([0-9]+)',l[i])
        if g(y) != 0:
            a = list(re.finditer('([0-9]+)',l[i]))
            c=l[i][:]
            for j in range(len(a)):
                c=c.replace(a[j].group(0),'',1)
            r.append(c)
        else:
            r.append(l[i])
        e.append(y)
    return e,r
    
document3 = """import json as a,sys,re
from functools import cmp_to_key
g=len
s=sorted
def p(x,y):
    d,b=g(x[0]),g(y[0])
    if x[1]==y[1]:
        if d-b:
            return 1 if d>b else -1
        else:
            for i in range(d):
                if x[0][i]==y[0][i]:
                    continue
                return 1 if int(x[0][i])>int(y[0][i]) else -1       
            return 0
    return 1 if x[1]>y[1] else -1
def t(l):
    r,e=[],[]
    for i in range(g(l)):
        y=re.findall('([0-9]+)',l[i])
        if g(y) != 0:
            d = list(re.finditer('([0-9]+)',l[i]))
            c=l[i][:]
            for j in range(len(d)):
                c=c.replace(d[j].group(0),'',1)
            r.append(c)
        else:
            r.append(l[i])
        e.append(y)
    return e,r
n,o=sys.argv[1:3]
k,v=a.load(open(n))
w,x=t(k)
y,z=t(v)
c=s(zip(w,x,k),key=cmp_to_key(p))
b=s(zip(y,z,v),key=cmp_to_key(p))
a.dump(dict([(c[i][2],b[i][2]) for i in range(g(c))]),open(o,'w'))"""

def extract_remove(lst):
    remove, extract = [], []
    for i in range(len(lst)):
        integers = re.findall('([0-9]+)',lst[i])
        if len(integers) != 0:
            integer_lst = list(re.finditer('([0-9]+)',lst[i]))
            cpy = lst[i][:]
            for j in range(len(integer_lst)):
                cpy = cpy.replace(integer_lst[j].group(0),'',1)
            remove.append(cpy)
        else:
            remove.append(lst[i])
        extract.append(integers)
    return extract, remove
def check_type(origin_mapping):
    flag = 0
    keys, values = zip(*origin_mapping.items())
    keys, values = sorted(keys), sorted(values)
    for i in range(len(keys)):
        if values[i] != origin_mapping[keys[i]]:
            flag = 1
            break
    if flag == 0:
        return 0
    #kv_lst = sorted(list(origin_mapping.items()),key=lambda x: x[0])
    kv_lst = list(origin_mapping.items())
    # divide kv_list to k and v
    k,v = zip(*kv_lst)
    k = list(k)
    v = list(v)   
    # ke: extraction of int of key
    # ve: extraction of int of value
    # kr: string removed int part of key
    # vr: string removed int part of value
    ke, kr = t(k)
    ve, vr = t(v)
    table = zip(ke,kr,ve,vr,k,v)
    table = sorted(table,key=cmp_to_key(p))      
    t0,t1,t2,t3,t4,t5 = zip(*table)
    check_table = zip(t2,t3,[i for i in range(len(k))])
    check_table = sorted(check_table,key=cmp_to_key(p))
    ccc = 0
    for i in range(len(k)):
        if check_table[i][2] != i:
            ccc += 1
            #print(table[i-2:i+1])
            #break
    print('shift: {}'.format(ccc))
    if ccc == 0:
        return 1
    

    print('using my own comparator')
    i0,i1,i2=zip(*check_table)
    #print('original idx list char count: {}'.format(len(str(list(i2)))))
    i2temp = consecu_int_run_length(list(i2))
    print('consecu idx list char count: {}'.format(len(str(i2temp))))
    i2temp1 = sporadic_seg_run_length(i2temp)
    print('sporadic_seg idx list char count: {}'.format(len(str(i2temp1))))
    print()
    
    print('using default comparator')
    mmm = sorted(list(origin_mapping.items()),key=lambda x: x[0])
    
    k,v = zip(*mmm)
    k = list(k)
    v = list(v)          
    mm = sorted(zip(v,[ i for i in range(len(k)) ]),key=lambda x: x[0])
    v, idx = zip(*mm) 
    idx = list(idx)
    mark = consecu_int_run_length(idx)
    print('consecu idx list char count: {}'.format(len(str(mark))))
    sporadic_seg = sporadic_seg_run_length(mark)
    print('sporadic_seg idx list char count: {}'.format(len(str(sporadic_seg))))
    print()
    return 2
if __name__ == '__main__':
    case = 1
    print('case: {}'.format(case))
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    nf = '../input/case{}_in.json'.format(case)
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()
    # rule based classifying
    same, prefix, infix, postfix, others = classify(origin_mapping)
   
    print('type: {}'.format(check_type(origin_mapping)))
    
       
    
    '''
    v, idx = zip(*mm) 
    
    idx = list(idx)
    v = list(v)
    mark = consecu_int_run_length(idx)
    print('len of str of mark: {}'.format(len(str(mark))))
    sporadic_seg = sporadic_seg_run_length(mark)
    print('len of str of sporadic_seg: {}'.format(len(str(sporadic_seg))))'''
    document1 = """import json as a,sys 
w={}
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
k,v=set(k),set(v)
r=k&v
c=list(r)
m=dict(zip(c,c))
k,v=s(k-r),s(v-r)
l=len
g=range
for i in g(l(k)):
    m[k[i]]=v[i]
for i in g(0,l(w),3):
    if w[i+2]!=1:
        for j in g(w[i+2]):
             m[k[w[i+1]+j]]=v[w[i]+j]
    else:
        for j in g(l(w[i+1])):
            m[k[w[i+1][j]]]=v[w[i]+j]
a.dump(m,open(o,'w'))""".format(str(sporadic_seg))

    document2 = b"""import json as a,sys
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
a.dump(dict(zip(s(k),s(v))),open(o,'w'))"""   

    #script=open(script_name,'w')
    #script.write(document1 if len(sporadic_seg) != 0 else document2)
    #script.close()

case: 1
same: 448
prefix: 225
infix: 0
postfix: 0
others: 10643
shift: 11031
using my own comparator
consecu idx list char count: 20118
sporadic_seg idx list char count: 17611

using default comparator
consecu idx list char count: 50161
sporadic_seg idx list char count: 46243

type: 2


In [524]:
string1 = 'ga_kc/ga_kc_chn_jt/Slickered_bud[1]/h0/i1/i1/l0'
string2 = 'ga_kc/ga_kc_chn_jt/Slickered_bud[2]/h0/i1/i1/l0'
'''integers = re.findall('([0-9]+)',lst[i])
        if len(integers) != 0:
            integer_lst = list(re.finditer('([0-9]+)',lst[i]))'''
dirs = re.findall('([^/]+)/',string1)
def p(x,y):
    d,b=g(x[0]),g(y[0])
    if x[1]==y[1]:
        if d-b:
            return 1 if d>b else -1
        else:
            for i in range(d):
                if x[0][i]==y[0][i]:
                    continue
                return 1 if int(x[0][i])>int(y[0][i]) else -1       
            return 0
    return 1 if x[1]>y[1] else -1
def rep(_,dim):
    for i in range(len(dim)):
        _ = _.replace(dim[i].group(0),'',1)
    return _
def int_cmp(rhs,lhs):
    return 1 if int(rhs)>int(lhs) else -1
def idx_cmp(rhs,lhs):
    for j in range(len(rhs)):
        rint = rhs[j].group(0)
        lint = lhs[j].group(0)
        if rint == lint:
            continue
        else:
            return int_cmp(rint,lint)
    return 0
def directory_based(rhs,lhs):
    rdirs = list(re.finditer('([^/]*)/',rhs[0]))
    ldirs = list(re.finditer('([^/]+)/',lhs[0]))
    dirss = len(ldirs) if len(rdirs) > len(ldirs) else len(rdirs)
    if dirss == 0:
        rtemp = rhs[0]
        ltemp = lhs[0]
        rdim = list(re.finditer('([0-9]+)',rtemp))
        ldim = list(re.finditer('([0-9]+)',ltemp))
        rtemp = rep(rtemp,rdim)
        ltemp = rep(ltemp,ldim)
        if rtemp == ltemp:
            if len(rdim) == len(ldim):
                return idx_cmp(rdim,ldim)
            else:
                return 1 if len(rdim)>len(ldim) else -1
        return 1 if rtemp>ltemp else -1
    for i in range(dirss):
        rtemp = rdirs[i].group(0)
        ltemp = ldirs[i].group(0)
        rdim = list(re.finditer('([0-9]+)',rtemp))
        ldim = list(re.finditer('([0-9]+)',ltemp))
        rtemp = rep(rtemp,rdim)
        ltemp = rep(ltemp,ldim)  
        if rtemp == ltemp:
            if len(rdim) == len(ldim):
                if idx_cmp(rdim,ldim) != 0:
                    return idx_cmp(rdim,ldim)
            else:
                return 1 if len(rdim)>len(ldim) else -1
            continue
        else:
            dimm = len(ldim) if len(rdim) > len(ldim) else len(rdim)
            rtemp = rdirs[i].group(0)
            ltemp = ldirs[i].group(0)
            rb = 0
            lb = 0
            for j in range(dimm):
                if rtemp[rb:rdim[j].start()] == ltemp[lb:ldim[j].start()]:
                    if rdim[j].group(0) == ldim[j].group(0):
                        rb = rdim[j].span()[1]
                        lb = ldim[j].span()[1]
                        continue
                    return int_cmp(rdim[j].group(0),ldim[j].group(0))
        return 1 if rtemp>ltemp else -1
    return 1 if len(rdirs)>len(ldirs) else -1
#print(directory_based(string1,string2))
print(len(getsource(directory_based)))
string3 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[0][0]_DR_tob_sai[0][1]_DR_tob_sai[0][2]_DR_tob_sai[0][3]/unta_$O1/lao1/O$1'
string4 = 'INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][8]_DR_tob_sai[5][15]/unta_$O1/lao1/O$1'
#print(directory_based(string3,string4))
#keys=sorted(keys,key=cmp_to_key(directory_based))
#values=sorted(values,key=cmp_to_key(directory_based))

rtable = {}
for kkk,vvv in origin_mapping.items():
    rtable[vvv] = kkk

mmm = list(rtable.items())
mmm=sorted(mmm,key=cmp_to_key(directory_based))
a0,a1 =zip(*mmm)
aaa = list(zip(a1,[i for i in range(len(mmm))]))
aaa=sorted(aaa,key=cmp_to_key(directory_based))
ccc = 0
for i in range(len(mmm)):
    if i != aaa[i][1]:
        ccc += 1
print(ccc)
for i in range(100,200,1):
    if rtable[a0[i]] == aaa[i][0]:
        print('A:',a0[i])
        print('B:',aaa[i][0])
    else:
        print('A:',a0[i])
        print('B:',aaa[i][0])
        print('*B should be',rtable[a0[i]])
    
v, idx = zip(*aaa) 
idx = list(idx)
mark = consecu_int_run_length(idx)
print('consecu idx list char count: {}'.format(len(str(mark))))
sporadic_seg = sporadic_seg_run_length(mark)
print('sporadic_seg idx list char count: {}'.format(len(str(sporadic_seg))))
print()

1811
3274
A: INCROSS_LAG_VIPS/tob_sai[5][2]/O$1
B: INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][0]_DR_tob_sai[5][1]_DR_tob_sai[5][2]_DR_tob_sai[5][3]/unta_$O3/lao1/O$1
A: INCROSS_LAG_VIPS/tob_sai[5][3]/O$1
B: INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][0]_DR_tob_sai[5][1]_DR_tob_sai[5][2]_DR_tob_sai[5][3]/unta_$O4/lao1/O$1
A: INCROSS_LAG_VIPS/tob_sai[5][4]/O$1
B: INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][4]_DR_tob_sai[5][5]_DR_tob_sai[5][6]_DR_tob_sai[5][7]/unta_$O1/lao1/O$1
A: INCROSS_LAG_VIPS/tob_sai[5][5]/O$1
B: INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][4]_DR_tob_sai[5][5]_DR_tob_sai[5][6]_DR_tob_sai[5][7]/unta_$O2/lao1/O$1
A: INCROSS_LAG_VIPS/tob_sai[5][6]/O$1
B: INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][4]_DR_tob_sai[5][5]_DR_tob_sai[5][6]_DR_tob_sai[5][7]/unta_$O3/lao1/O$1
A: INCROSS_LAG_VIPS/tob_sai[5][7]/O$1
B: INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][4]_DR_tob_sai[5][5]_DR_tob_sai[5][6]_DR_tob_sai[5][7]/unta_$O4/lao1/O$1
A: INCROSS_LAG_VIPS/tob_sai[5][8]/O$1
B: INCROSS_LAG_VIPS/TIU_FIGS_tob_sai[5][8]_DR_tob_sa